In [1]:
import numpy as np
import pandas as pd
import functions.multi_occupation_network as multi_occupation_network

# Calibration Excersize 2: One Occupation Per Sector

Suppose there is one type of labor specific to each sector, in other words that $\mathcal{O}=J$. We calibrate our model to mathc BEA input-output tables, and treat the residual, the part of production not accounted for by intermediate inputs, as the labor elasticity. 

In [2]:
data_dir = '../data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_yearly= pd.read_csv(data_dir + 'labor_market_yearly.csv')
dfLabor_market_yearly = dfLabor_market_yearly.sort_values(by=['year', 'BEA_sector'])
dfLabor_market_yearly = dfLabor_market_yearly.dropna(axis=0)
dfLabor_market_yearly = dfLabor_market_yearly[dfLabor_market_yearly['year'] == 2005]
# reformatting parameters
Omega = np.array(dfA.iloc[:, 1:], dtype='float64')
Psi = np.linalg.inv(np.eye(Omega.shape[0])-Omega)
J = Omega.shape[0]
O = J

φ = np.array(dfParam.φ)

epsN = np.diag(np.array(dfParam.α))
epsD = np.array(dfParam.θ_alt).reshape((J,1))

θ = dfLabor_market_yearly['v'] / dfLabor_market_yearly['u'] 
ν = np.ones(J) * 0.5
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) + curlyQ
r = 0.045
curlyT = np.diag(r / (φ*θ**(-ν) - r))
# NOTE: here I am picking the r so the τ's roughly look right. Down the line, we can calibrate τ directly from the data. 
# Also note that tightness is computed from unemployed workers, but in our model it should be computed from the whole work force.
curlyL = np.eye(J)

First, we define the shocks we are interested in to allow us to easily change these in the future, that is we define the vectors $d\log\bm{A}$ and $d\log\bm{H}$.

In [3]:
dlog_A = np.zeros((J, 1))
dlog_A[3] = 0.01 
dlog_H = np.zeros((O,1))

We assume Cobb-Douglas production and preferences.

In [4]:
dlog_lam = np.zeros_like(dlog_A)
dlog_epsN = np.zeros_like(epsN)
dlog_epsD = np.zeros_like(epsD)

Finally, we need to make some assumption about how wages change. We begin by assuming that nominal wages do not change in respond to changes in either productivity or the labor force.

In [5]:
epsW_A = np.zeros((O,J))
epsW_H = np.zeros((O,O))
dlog_w = multi_occupation_network.WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)

With these pieces in hand, we can now estimate responses to tightness. For details, see the multiple occupations example code.

In [11]:
curlyE = multi_occupation_network.curlyEFunc(dlog_epsN,epsN)
num=0
dlog_theta = multi_occupation_network.ThetaFunc(dlog_A, dlog_H, dlog_w, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN, num=num)
dlog_theta

array([[0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312],
       [0.12895312]])

In [12]:
dlog_p = multi_occupation_network.PriceFunc(dlog_A, dlog_w, dlog_theta, Psi, curlyQ, epsN, curlyT)
dlog_p

array([[-4.39427141e-15],
       [-1.01573258e-04],
       [-6.56725591e-03],
       [-1.91842683e-02],
       [ 6.57029691e-04],
       [ 2.42509693e-03],
       [ 6.91665896e-04],
       [ 1.08402680e-03],
       [-3.18343466e-04],
       [-2.62427516e-03],
       [-2.47735813e-03],
       [-7.08478726e-04],
       [ 4.75894022e-04],
       [-6.68593750e-04],
       [ 1.74571192e-04],
       [-3.93469975e-04],
       [ 7.77278059e-04]])

In [13]:
dlog_y = multi_occupation_network.OutputFunc(dlog_A,dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.06447656],
       [0.06457813],
       [0.07104382],
       [0.08366083],
       [0.06381953],
       [0.06205146],
       [0.06378489],
       [0.06339253],
       [0.0647949 ],
       [0.06710084],
       [0.06695392],
       [0.06518504],
       [0.06400067],
       [0.06514515],
       [0.06430199],
       [0.06487003],
       [0.06369928]])

In [14]:
dlog_aggY = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
dlog_aggY

array([0.06924625, 0.06924625, 0.06924625, 0.06924625, 0.06924625,
       0.06924625, 0.06924625, 0.06924625, 0.06924625, 0.06924625,
       0.06924625, 0.06924625, 0.06924625, 0.06924625, 0.06924625,
       0.06924625, 0.06924625])

In [10]:
epsD.T @ Psi @ dlog_A

array([0.00717491])